In [54]:
import nltk,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import nltk.data
import csv
from nltk.stem.porter import *
pd.options.mode.chained_assignment = None
from sklearn.preprocessing import LabelEncoder
from numpy import array as arr
from numpy import argmax

In [55]:
#Read data from dataset
#data = pd.read_csv("mbti_1.csv") 
#shortdata=data.iloc[:,-1]
arr=[]
file = open("downloaded1.csv",'rt')
samples=csv.reader(file)
c=0
for i in samples:
    c+=1
    
    if c==2:
        x=i[1]
        break

for i in samples:
    if i[1]!=x:
        arr.append(i)

data=arr
df=pd.DataFrame(data=arr,columns=("types","posts"))
#print(len(df.columns))
#print(df)
#shortdata=shortdata.head()
#shortdata1=data.iloc[0:5,0]
#print('-----Data-------')
#print(shortdata)

In [56]:
def labelencode(df):
    data=df['types']
    values=np.array(data)
    label=LabelEncoder()
    intencode=label.fit_transform(values)
    df['typeint']=intencode
    print(list(label.inverse_transform([0,1,2])))
    #df['typeint'].plot(kind='hist')
    #k=np.arange(0,16)
    #x=label.inverse_transform(k)   #can access encoded actual value using x
    #print(values)
    return df

df=labelencode(df)
#print(df)
shortdata=df.iloc[:,1]
#print(shortdata)

['negative', 'neutral', 'positive']


In [57]:
#removing stopwords 
from nltk.corpus import stopwords
stop=stopwords.words("english")
print('------Removing stopwords------')
shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word!='i' or word!='I']))
#print(shortdata)
#stemming of words
ps = PorterStemmer()
print('-------Stemming--------')
shortdata = shortdata.apply(lambda x: ' '.join([ps.stem(word) for word in x.split() ]))
#print(shortdata)

------Removing stopwords------
-------Stemming--------


In [58]:
#removing non-alphabets
shortdata=shortdata.apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
#print(shortdata)
print('-------Lemmatization--------')
shortdata = shortdata.apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split() ]))
#print(shortdata)

print('--------Removing punctuations--------')
def clear_punctuation(s):
	import string
	#print("\n")
	clear_string = ""
	for symbol in s:
		if symbol not in string.punctuation:
			clear_string += symbol
	return clear_string

shortdata = shortdata.apply(lambda x: ''.join(clear_punctuation(x))  )
#print(shortdata)
#for line in shortdata:
#	print(line)
#	print('\n')


-------Lemmatization--------
--------Removing punctuations--------


In [59]:
def strip_all_entities(text):
	import string
	entity_prefixes = ['@']
	for separator in  string.punctuation:
		if separator not in entity_prefixes :
			text = text.replace(separator,' ')
	words = []
	for word in text.split():
		word = word.strip()
		if word:
			if word[0] not in entity_prefixes:
				words.append(word)
	return ' '.join(words)

shortdata = shortdata.apply(lambda x: ''.join(strip_all_entities(x))  ) 



In [60]:
i=0
arr=[]
print("-----PREPROCESSED_DATA------")
count=0
for line in shortdata:
    df.iloc[i,1]=line
    i=i+1
#print(df)



'''for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])'''

-----PREPROCESSED_DATA------


'for column in dataset.columns:\n    if dataset[column].dtype == type(object):\n        le = LabelEncoder()\n        dataset[column] = le.fit_transform(dataset[column])'

In [116]:
proc_data=np.array(df['posts'])
label=np.array(df['typeint'])
print(len(proc_data))
print(len(label))

6936
6936


In [117]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(proc_data, label, test_size=0.2)

In [118]:
print(train_x)

['joe road monday tuesday due impend stay safe hope see wednesday'
 'mark good weekend statu lazi sunday'
 'okami releas psn wait play To one greatest game ever' ...
 'internship websit launch nebraska governor dave heineman today announc launch new'
 'may lower rank mcgeezer go D leagu'
 'arsen v new sign loui saha set lead line absenc stephan']


In [119]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

# only work with the 3000 most popular words found in our dataset
max_words = 20000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories


In [120]:
train_y = keras.utils.to_categorical(train_y, 3)

In [121]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(500, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
sgd=keras.optimizers.SGD(lr=0.5, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='categorical_crossentropy',
  optimizer=sgd,
  metrics=['accuracy'])
model.summary()

In [163]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 500)               10000500  
_________________________________________________________________
dropout_6 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 1503      
Total params: 10,002,003
Trainable params: 10,002,003
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

Train on 4993 samples, validate on 555 samples
Epoch 1/5
4993/4993 [==============================] - 10s 2ms/step - loss: 0.9198 - acc: 0.5722 - val_loss: 1.5649 - val_acc: 0.3622
Epoch 2/5
4993/4993 [==============================] - 9s 2ms/step - loss: 0.7653 - acc: 0.6649 - val_loss: 1.5648 - val_acc: 0.3874
Epoch 3/5
4993/4993 [==============================] - 9s 2ms/step - loss: 0.6369 - acc: 0.7276 - val_loss: 2.9717 - val_acc: 0.1495
Epoch 4/5
4993/4993 [==============================] - 9s 2ms/step - loss: 0.5411 - acc: 0.7739 - val_loss: 1.5223 - val_acc: 0.5748
Epoch 5/5
4993/4993 [==============================] - 9s 2ms/step - loss: 0.4185 - acc: 0.8404 - val_loss: 0.9625 - val_acc: 0.5838


In [123]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

In [124]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=max_words)
# for human-friendly printing
labels = ['negative','neutral','positive']

# read in our saved dictionary
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

# read in your saved model structure
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights('model.h5')

# okay here's the interactive part


In [133]:
import string
def preprocess(a):
    from nltk.stem.wordnet import WordNetLemmatizer
    lmtzr = WordNetLemmatizer()
    sentence=a
    from nltk.corpus import stopwords
    ps = PorterStemmer()
    stop=stopwords.words("english")
    
    b=sentence.split()
    for i in sentence.split():
        if i in stop:
            b.remove(i)
    for i in b:
        ps.stem(i)
        lmtzr.lemmatize(i,'v')
    c=""
    for i in b:
        
        c=c+i+" "
   
    clear_string = ""
    for symbol in c:
        if symbol not in string.punctuation:
            clear_string += symbol
    return (strip_all_entities(clear_string))
    

In [195]:
eval=input("enter a tweet:")
eval=preprocess(eval) 
testArr = convert_text_to_index_array(eval)
inp = tokenizer.sequences_to_matrix([testArr], mode='binary')
# predict which bucket your input belongs ini
pred = model.predict(inp)
# and print it for the humons
print("%s sentiment; %f%% confidence" % (labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))

enter a tweet:hjg
'hjg' not in training corpus; ignoring.
neutral sentiment; 48.543015% confidence


In [40]:
print(test_x)

['An even maya angel elon alumni ticket avail'
 'activist demand releas jharkhand tribal new nov condemn arrest woman'
 'thing know the postal brink default' ...
 'come anoth chanc win ticket tomorrow hudson'
 'I debat total follow poetic'
 'final watch last week true readi new episod how everyon els']


In [41]:
que=[]
for i in test_x:
    #evalSentence = input('Input a sentence to be evaluated, or Enter to quit: ')
    evalSentence=i
    if len(evalSentence) == 0:
        break

    # format your input for the neural net
    testArr = convert_text_to_index_array(evalSentence)
    inp = tokenizer.sequences_to_matrix([testArr], mode='binary')
    # predict which bucket your input belongs in
    pred = model.predict(inp)
    k=np.amax(pred[0])
    print(pred[0])
    print(k)
    ind=list(pred[0]).index(k)
    print(labels[ind])
    que.append(ind)
    # and print it for the humons
    #print("%s sentiment; %f%% confidence" % (labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))

[0.07477323 0.8344759  0.09075097]
0.8344759
neutral
'jharkhand' not in training corpus; ignoring.
'tribal' not in training corpus; ignoring.
[0.01922852 0.92754483 0.05322655]
0.92754483
neutral
'brink' not in training corpus; ignoring.
[0.11738881 0.48436365 0.3982475 ]
0.48436365
neutral
[0.01013837 0.92644894 0.0634126 ]
0.92644894
neutral
[0.00501485 0.08917828 0.9058069 ]
0.9058069
positive
[0.1364079  0.7307445  0.13284773]
0.7307445
neutral
[0.4513676  0.27206016 0.27657232]
0.4513676
negative
[0.01029682 0.14266662 0.84703654]
0.84703654
positive
'carlo' not in training corpus; ignoring.
'salzburg' not in training corpus; ignoring.
[2.7387083e-04 5.1244698e-04 9.9921370e-01]
0.9992137
positive
'disappear' not in training corpus; ignoring.
[0.42240983 0.38203117 0.19555904]
0.42240983
negative
'makin' not in training corpus; ignoring.
[0.5514454  0.04035152 0.408203  ]
0.5514454
negative
[0.14305294 0.16742744 0.6895196 ]
0.6895196
positive
[0.0156099  0.04310099 0.9412891 ]
0.

[0.02305406 0.34192118 0.6350247 ]
0.6350247
positive
'rite' not in training corpus; ignoring.
'hhave' not in training corpus; ignoring.
'domest' not in training corpus; ignoring.
[0.11006673 0.72716874 0.16276447]
0.72716874
neutral
'youv' not in training corpus; ignoring.
'squadron' not in training corpus; ignoring.
[0.01918792 0.9484966  0.03231555]
0.9484966
neutral
'crucifi' not in training corpus; ignoring.
'rome' not in training corpus; ignoring.
[0.05084695 0.5037767  0.4453763 ]
0.5037767
neutral
[1.2230585e-04 1.3844373e-03 9.9849319e-01]
0.9984932
positive
[0.01692513 0.91962415 0.06345081]
0.91962415
neutral
[0.03780293 0.8479089  0.11428822]
0.8479089
neutral
'omaha' not in training corpus; ignoring.
[0.04641173 0.63699627 0.31659204]
0.63699627
neutral
[0.00115504 0.96146786 0.0373771 ]
0.96146786
neutral
'milanello' not in training corpus; ignoring.
[0.04582788 0.8210743  0.13309784]
0.8210743
neutral
'ganda' not in training corpus; ignoring.
'pagkafriend' not in trainin

[0.02828627 0.21428308 0.7574307 ]
0.7574307
positive
[0.00213764 0.00837511 0.9894873 ]
0.9894873
positive
'bait' not in training corpus; ignoring.
[0.25297365 0.49821764 0.24880867]
0.49821764
neutral
'dalken' not in training corpus; ignoring.
'rbi' not in training corpus; ignoring.
'rf' not in training corpus; ignoring.
[0.08528238 0.4505664  0.46415117]
0.46415117
positive
'furi' not in training corpus; ignoring.
'brink' not in training corpus; ignoring.
'tyson' not in training corpus; ignoring.
'furi' not in training corpus; ignoring.
'boytsov' not in training corpus; ignoring.
[0.0966185  0.88814646 0.015235  ]
0.88814646
neutral
'pietrasanta' not in training corpus; ignoring.
[2.6415853e-04 5.6614373e-03 9.9407440e-01]
0.9940744
positive
'paolo' not in training corpus; ignoring.
'canio' not in training corpus; ignoring.
[0.46563438 0.14207308 0.3922925 ]
0.46563438
negative
'groceri' not in training corpus; ignoring.
'pokemon' not in training corpus; ignoring.
'purchas' not in t

[0.1498402  0.7793073  0.07085243]
0.7793073
neutral
[0.14709091 0.15157364 0.70133543]
0.70133543
positive
'martinez' not in training corpus; ignoring.
'alburquerqu' not in training corpus; ignoring.
[0.02306437 0.39123273 0.58570284]
0.58570284
positive
'maidana' not in training corpus; ignoring.
[0.8987521  0.08374581 0.01750213]
0.8987521
negative
[0.01798358 0.137608   0.8444084 ]
0.8444084
positive
[0.30730122 0.2255703  0.46712843]
0.46712843
positive
[0.10307035 0.7507739  0.14615573]
0.7507739
neutral
'refrain' not in training corpus; ignoring.
[0.07489794 0.86267745 0.06242462]
0.86267745
neutral
[0.12889376 0.86736    0.00374631]
0.86736
neutral
[1.4889333e-03 4.2247539e-04 9.9808854e-01]
0.99808854
positive
'halal' not in training corpus; ignoring.
'halal' not in training corpus; ignoring.
[0.77144307 0.14324695 0.08530989]
0.77144307
negative
[0.00536263 0.02913561 0.9655018 ]
0.9655018
positive
'hump' not in training corpus; ignoring.
'iker' not in training corpus; ignori

[0.67564774 0.25654504 0.06780724]
0.67564774
negative
[0.14015    0.70305306 0.156797  ]
0.70305306
neutral
[0.00382408 0.06658006 0.9295959 ]
0.9295959
positive
'captiv' not in training corpus; ignoring.
'stantonburi' not in training corpus; ignoring.
[0.04760739 0.9058931  0.04649959]
0.9058931
neutral
'reasonabley' not in training corpus; ignoring.
[0.12322403 0.51837415 0.3584018 ]
0.51837415
neutral
'coventri' not in training corpus; ignoring.
[0.09207515 0.69790643 0.21001846]
0.69790643
neutral
'giroud' not in training corpus; ignoring.
[0.02697751 0.7412071  0.23181534]
0.7412071
neutral
'earl' not in training corpus; ignoring.
[0.14484352 0.70131814 0.15383841]
0.70131814
neutral
'curto' not in training corpus; ignoring.
'fairli' not in training corpus; ignoring.
[0.48076653 0.483097   0.03613648]
0.483097
neutral
[0.38605326 0.53359175 0.08035499]
0.53359175
neutral
'vanilla' not in training corpus; ignoring.
'bean' not in training corpus; ignoring.
'chandler' not in trainin

...

[1.03465965e-04 3.64213765e-01 6.35682762e-01]
0.63568276
positive
[0.0239481  0.03933964 0.9367123 ]
0.9367123
positive
[0.10313054 0.2452407  0.6516288 ]
0.6516288
positive
[0.05077529 0.46282518 0.48639953]
0.48639953
positive
[0.01416278 0.1935454  0.79229176]
0.79229176
positive
'storytel' not in training corpus; ignoring.
[6.4806663e-05 1.9243649e-01 8.0749869e-01]
0.8074987
positive
[0.02278717 0.13813792 0.8390749 ]
0.8390749
positive
[0.5032526  0.15831146 0.33843595]
0.5032526
negative
[0.41222596 0.40568215 0.18209188]
0.41222596
negative
'meanwhil' not in training corpus; ignoring.
'criticis' not in training corpus; ignoring.
'slightli' not in training corpus; ignoring.
[0.03648388 0.896115   0.06740105]
0.896115
neutral
'pen' not in training corpus; ignoring.
[0.4826138  0.39986983 0.11751639]
0.4826138
negative
[4.7990552e-04 2.0839458e-03 9.9743623e-01]
0.9974362
positive
[0.00825083 0.9726839  0.01906517]
0.9726839
neutral
'kansa' not in training corpus; ignoring.
'spor

[1.9806968e-04 3.2007003e-03 9.9660122e-01]
0.9966012
positive
'invis' not in training corpus; ignoring.
[0.00588475 0.70964396 0.28447133]
0.70964396
neutral
'onstag' not in training corpus; ignoring.
[0.02726989 0.71896285 0.25376725]
0.71896285
neutral
'jinja' not in training corpus; ignoring.
'safari' not in training corpus; ignoring.
'opossom' not in training corpus; ignoring.
[0.00415726 0.9775653  0.0182775 ]
0.9775653
neutral
'romania' not in training corpus; ignoring.
[0.08451673 0.37860066 0.5368826 ]
0.5368826
positive
[0.6634804  0.21179    0.12472957]
0.6634804
negative
'hubbi' not in training corpus; ignoring.
[0.11052608 0.7111064  0.17836753]
0.7111064
neutral
[0.00498198 0.08000146 0.91501653]
0.91501653
positive
'mo' not in training corpus; ignoring.
'interchang' not in training corpus; ignoring.
'tilt' not in training corpus; ignoring.
[0.02668878 0.9500015  0.02330969]
0.9500015
neutral
[0.0404703  0.68411565 0.27541402]
0.68411565
neutral
'excess' not in training c

[0.00206757 0.03069974 0.9672327 ]
0.9672327
positive
[0.01854366 0.17459743 0.8068589 ]
0.8068589
positive
[0.24921194 0.48125392 0.2695342 ]
0.48125392
neutral
'ident' not in training corpus; ignoring.
[0.0299936  0.91245264 0.05755382]
0.91245264
neutral
[0.00479802 0.959169   0.036033  ]
0.959169
neutral
[0.11271815 0.37927383 0.50800806]
0.50800806
positive
[0.29855838 0.2816738  0.41976786]
0.41976786
positive
[0.00629277 0.85720795 0.13649923]
0.85720795
neutral
[0.21280086 0.6412299  0.14596924]
0.6412299
neutral
[0.01243217 0.14349914 0.8440687 ]
0.8440687
positive
'steelseri' not in training corpus; ignoring.
'illumin' not in training corpus; ignoring.
[0.06093026 0.49560434 0.4434654 ]
0.49560434
neutral
[0.19874524 0.34952343 0.45173135]
0.45173135
positive
[0.00709861 0.03115834 0.961743  ]
0.961743
positive
[0.0454208  0.20021072 0.7543684 ]
0.7543684
positive
[7.947913e-05 3.148517e-05 9.998890e-01]
0.999889
positive
[0.06699627 0.9062312  0.02677246]
0.9062312
neutral
'

[0.22435306 0.37765136 0.39799562]
0.39799562
positive
'environ' not in training corpus; ignoring.
'agenc' not in training corpus; ignoring.
'oxon' not in training corpus; ignoring.
'waterway' not in training corpus; ignoring.
[0.0433704  0.7912689  0.16536064]
0.7912689
neutral
'alphabet' not in training corpus; ignoring.
[0.4694147  0.28103608 0.2495492 ]
0.4694147
negative
[0.23774756 0.50565505 0.25659737]
0.50565505
neutral
[0.00807697 0.01796257 0.9739604 ]
0.9739604
positive
[2.5752906e-04 9.9387580e-01 5.8666472e-03]
0.9938758
neutral
'reynold' not in training corpus; ignoring.
'prequel' not in training corpus; ignoring.
[0.10005531 0.7916514  0.10829325]
0.7916514
neutral
[0.180114   0.78047    0.03941596]
0.78047
neutral
'lat' not in training corpus; ignoring.
[0.05467447 0.0151787  0.9301468 ]
0.9301468
positive
[0.0323892  0.13627918 0.8313316 ]
0.8313316
positive
'instinct' not in training corpus; ignoring.
[0.00218082 0.01148503 0.98633415]
0.98633415
positive
'unmask' no

[1.7991741e-04 7.2993804e-03 9.9252069e-01]
0.9925207
positive
[0.12170142 0.3043727  0.57392585]
0.57392585
positive
[0.0198734  0.07930182 0.9008247 ]
0.9008247
positive
[0.00268349 0.26179522 0.7355213 ]
0.7355213
positive
[0.01389583 0.12089109 0.8652131 ]
0.8652131
positive
[0.16983621 0.43652838 0.39363536]
0.43652838
neutral
'cornwal' not in training corpus; ignoring.
'padstow' not in training corpus; ignoring.
'chip' not in training corpus; ignoring.
[0.42588672 0.24904348 0.3250698 ]
0.42588672
negative
[0.00339161 0.12660144 0.870007  ]
0.870007
positive
'custard' not in training corpus; ignoring.
[0.06591088 0.7584697  0.17561947]
0.7584697
neutral
'govt' not in training corpus; ignoring.
[0.29276255 0.6084925  0.09874494]
0.6084925
neutral
'dima' not in training corpus; ignoring.
[2.6739046e-06 4.3637516e-05 9.9995363e-01]
0.9999536
positive
'grappl' not in training corpus; ignoring.
'corrupt' not in training corpus; ignoring.
[0.00622766 0.84570783 0.14806457]
0.84570783
n

In [42]:
count=0
notp=0
notn=0
notq=0
for i in range(len(que)):
    if que[i]==test_y[i]:
        count+=1
print(count/len(que))
    

0.6195965417867435


In [43]:
newdf=pd.DataFrame(data=data,columns=("types","posts"))


In [44]:
newdf=labelencode(newdf)

['negative', 'neutral', 'positive']


In [45]:
nposts=np.array(newdf['posts'])
nlabels=np.array(newdf['typeint'])
print(len(nposts))
print(len(nlabels))

6936
6936


In [46]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [47]:
newdf['posts'] = newdf['posts'].apply(lambda x: x.lower())

newdf['posts'] = newdf['posts'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [48]:
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(newdf['posts'].values)
X = tokenizer.texts_to_sequences(newdf['posts'].values)
X = sequence.pad_sequences(X)

In [49]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 183, 128)          640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 591       
Total params: 895,391
Trainable params: 895,391
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
Y = pd.get_dummies(newdf['types']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5548, 183) (5548, 3)
(1388, 183) (1388, 3)


In [51]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 30s - loss: 0.9520 - acc: 0.5283
Epoch 2/7
 - 33s - loss: 0.6794 - acc: 0.7172
Epoch 3/7
 - 32s - loss: 0.4492 - acc: 0.8282
Epoch 4/7
 - 32s - loss: 0.3067 - acc: 0.8911
Epoch 5/7
 - 32s - loss: 0.2205 - acc: 0.9198
Epoch 6/7
 - 32s - loss: 0.1584 - acc: 0.9483
Epoch 7/7
 - 35s - loss: 0.1075 - acc: 0.9663


In [52]:

score,acc=(model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size))
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.85
acc: 0.61
